In [0]:
%pylab inline
import xarray as xr
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Populating the interactive namespace from numpy and matplotlib
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Set your directory path
#YOUR_DIRECTORY = '/content/drive/My Drive/ATMS597 Weather Climate Data Analysis/Module 6/Project 6/'  # Chu-Chun
YOUR_DIRECTORY = '/content/drive/My Drive/Colab Notebooks/Project_6/' # Jeff

In [60]:
# Load MJO data
MJOIndices_df = pd.read_csv(YOUR_DIRECTORY + 'MJOIndices_JMA_1980to2020.csv', index_col = 'Unnamed: 0')
MJOIndices_df.columns.name = 'Date'
MJOIndices_df

Date,MJOPhase_OLR,MJOLongitude_OLR,MJOAmplitude_OLR,MJOPhase_Chi200,MJOLongitude_Chi200,MJOAmplitude_Chi200
1980-01-01,5,148.6,0.6049,5,175.4,0.5459
1980-01-02,5,164.0,0.6069,6,187.0,0.7355
1980-01-03,5,176.0,0.5613,6,198.4,0.6891
1980-01-04,6,185.2,0.6842,6,191.3,0.7448
1980-01-05,6,199.9,0.8435,6,215.7,0.8425
...,...,...,...,...,...,...
2020-02-15,5,176.4,1.6450,5,180.0,1.6147
2020-02-16,5,171.4,1.7534,5,167.1,1.7944
2020-02-17,5,164.4,1.7153,5,166.2,1.9357
2020-02-18,5,163.8,1.6732,5,172.0,1.8460


In [61]:
# Load Bahia Blanca rainfall data
BB_rainfall = pd.read_csv(YOUR_DIRECTORY + 'ghcn_data.csv')

# Fill in missing days with NaN
idx = pd.date_range('1980-01-01', '2019-12-31')
BB_rainfall.index = pd.to_datetime(BB_rainfall['date'])
BB_rainfall = BB_rainfall.reindex(idx, fill_value = NaN)
BB_rainfall

,date,datatype,station,attributes,value
1980-01-01,1980-01-01,PRCP,GHCND:AR000877500,",,S,",0.0
1980-01-02,1980-01-02,PRCP,GHCND:AR000877500,",,S,",0.0
1980-01-03,1980-01-03,PRCP,GHCND:AR000877500,",,S,",0.0
1980-01-04,1980-01-04,PRCP,GHCND:AR000877500,",,S,",0.0
1980-01-05,1980-01-05,PRCP,GHCND:AR000877500,",,S,",18.0
...,...,...,...,...,...
2019-12-27,NaN,NaN,NaN,NaN,NaN
2019-12-28,2019-12-28,PRCP,GHCND:AR000877500,",,S,",0.0
2019-12-29,NaN,NaN,NaN,NaN,NaN
2019-12-30,2019-12-30,PRCP,GHCND:AR000877500,",,S,",5.1


In [62]:
# Load ERA5 soil moisture data
data1 = xr.open_dataset(YOUR_DIRECTORY + 'ERA5_swvl1.nc')
data2 = xr.open_dataset(YOUR_DIRECTORY + 'ERA5_swvl2.nc')
data2

<xarray.Dataset>
Dimensions:    (latitude: 2, longitude: 2, time: 14610)
Coordinates:
  * longitude  (longitude) float32 -62.25 -62.0
  * latitude   (latitude) float32 -38.5 -38.75
  * time       (time) datetime64[ns] 1980-01-01T12:00:00 ... 2019-12-31T12:00:00
Data variables:
    swvl2      (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-05-03 15:50:32 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [63]:
swvl1_aave = np.mean(data1['swvl1'], axis=(1, 2))
swvl2_aave = np.mean(data2['swvl2'], axis=(1, 2))
ERA5_soil_moisture1_df = swvl1_aave.to_dataframe()
ERA5_soil_moisture2_df = swvl2_aave.to_dataframe()
ERA5_soil_moisture1_df.index = ERA5_soil_moisture1_df.index.strftime('%Y-%m-%d')
ERA5_soil_moisture2_df.index = ERA5_soil_moisture2_df.index.strftime('%Y-%m-%d')
ERA5_soil_moisture2_df

,swvl2
time,
1980-01-01,0.205773
1980-01-02,0.198426
1980-01-03,0.192707
1980-01-04,0.179462
1980-01-05,0.175685
...,...
2019-12-27,0.143121
2019-12-28,0.142428
2019-12-29,0.140815


In [0]:
# Add Rainfall data in the dataframe
MJOIndices_df['BahiaBlanca_Rainfall'] = BB_rainfall['value']

# Add ERA5 soil moisture data in the dataframe
MJOIndices_df['ERA5_Soil_Moisture_Lev1'] = ERA5_soil_moisture1_df
MJOIndices_df['ERA5_Soil_Moisture_Lev2'] = ERA5_soil_moisture2_df

# Removing rows for 2020
MJOIndices_df.index = pd.to_datetime(MJOIndices_df.index)
MJOIndices_df['Year'] = MJOIndices_df.index.year
MJOIndices_df = MJOIndices_df[MJOIndices_df['Year'] != 2020]
MJOIndices_df = MJOIndices_df.drop(columns = 'Year')

# Change column order
MJOIndices_df = MJOIndices_df[['BahiaBlanca_Rainfall', 'MJOPhase_OLR', 'MJOLongitude_OLR', 'MJOAmplitude_OLR',
                               'MJOPhase_Chi200', 'MJOLongitude_Chi200', 'MJOAmplitude_Chi200',
                               'ERA5_Soil_Moisture_Lev1', 'ERA5_Soil_Moisture_Lev2']]
MJOIndices_df.dropna()
#MJOIndices_df.to_csv(YOUR_DIRECTORY + 'SouthAmericanRainfall_MJOIndices_v1.csv')